# Homework 9


In [10]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import scipy.stats
import sys
from scipy.stats import skew


In [2]:
def simulate(A=1, B=1, C=10, D=1000):
  W = np.random.normal(0,1,D)
  X = W+np.random.normal(0,B,D)
  Y = A*X-W+np.random.normal(0,C,D)
  return Y, X, W

In [3]:
def run_trials(n_trials=1000, A=1, B=1, C=10, D=1000):
    count = 0
    for _ in range(n_trials):
        Y, X, W = simulate(A, B, C, D)
        XW = np.column_stack([X, W])
        XW = sm.add_constant(XW)
        model = sm.OLS(Y, XW).fit()
        t_val = model.tvalues[1]  # coefficient for X
        if abs(t_val) > 1.96:
            count += 1
    return count / n_trials

if __name__ == "__main__":
    prob = run_trials(n_trials=500, A=1, B=1, C=10, D=1000)
    print(f"Estimated probability: {prob:.3f}")

Estimated probability: 0.878


In [ ]:

def estimate_skew(n_trials=500, A=1, B=1, C=10, D=1000):
    coefs = []
    for _ in range(n_trials):
        Y, X, W = simulate(A, B, C, D)
        XW = sm.add_constant(np.column_stack([X, W]))
        model = sm.OLS(Y, XW).fit()
        coefs.append(model.params[1])  # coefficient for X
    return skew(coefs)

sk = estimate_skew(n_trials=1000000, A=1, B=1, C=10, D=1000)
print(f"Skewness of beta_X estimates: {sk:.4f}")

Skewness of beta_X estimates: 0.0022


In [11]:

# --- Monte Carlo Simulation ---

# Number of simulations to build the sampling distribution
num_simulations = 50000

# List to store the coefficient (beta_1) estimates from each simulation
beta_1_estimates = []

# Set a random seed for reproducible results
np.random.seed(42)

print(f"Running {num_simulations} simulations...")

for i in range(num_simulations):
    # 1. Run the simulation to get one dataset
    # We use the default parameters (A=1, B=1, C=10, D=1000)
    Y, X, W = simulate()

    # 2. Prepare the design matrix for OLS
    # We fit the model Y = beta_0 + beta_1*X
    # We need a column of ones for the intercept (beta_0)
    X_design = np.vstack([np.ones(len(X)), X]).T

    # 3. Calculate the OLS estimates using np.linalg.lstsq
    # This finds the coefficients [beta_0, beta_1] that minimize
    # the sum of squared residuals.
    # [0] access the coefficients array
    # [1] accesses the second coefficient (beta_1)
    beta_1 = np.linalg.lstsq(X_design, Y, rcond=None)[0][1]

    # 4. Store the estimate
    beta_1_estimates.append(beta_1)

print("Simulations complete.")

# --- Analyze the Distribution of Estimates ---

# 5. Calculate the skewness of the distribution of the estimates
# This tells us if the sampling distribution is symmetric (skew=0)
# or lopsided (skew > 0 or skew < 0)
skew_of_estimates = scipy.stats.skew(beta_1_estimates)

print(f"Skew of the {num_simulations} beta_1 estimates: {skew_of_estimates:.8f}")

# (Optional) We can also check the mean of the distribution
mean_of_estimates = np.mean(beta_1_estimates)
print(f"Mean of the {num_simulations} beta_1 estimates: {mean_of_estimates:.8f}")

Running 50000 simulations...
Simulations complete.
Skew of the 50000 beta_1 estimates: 0.01613101
Mean of the 50000 beta_1 estimates: 0.49927713


In [16]:
# Run trials for a given B
def detection_prob(B, n_trials=500, A=1, C=10, D=1000):
    count = 0
    for _ in range(n_trials):
        Y, X, W = simulate(A, B, C, D)
        XW = sm.add_constant(np.column_stack([X, W]))
        model = sm.OLS(Y, XW).fit()
        t_val = model.tvalues[1]  # coefficient for X
        if abs(t_val) > 1.96:
            count += 1
    return count / n_trials

# Search for B that gives ~50% detection
B_values = np.linspace(0.55, 0.65, 100)  # candidate B values
probs = [detection_prob(B) for B in B_values]

for B, p in zip(B_values, probs):
    print(f"B={B:.4f}, detection probability={p:.3f}")

# Find closest to 0.5
closest_idx = np.argmin(np.abs(np.array(probs) - 0.5))
print(f"Closest B ≈ {B_values[closest_idx]:.4f}, detection probability={probs[closest_idx]:.3f}")

B=0.5500, detection probability=0.404
B=0.5510, detection probability=0.438
B=0.5520, detection probability=0.416
B=0.5530, detection probability=0.432
B=0.5540, detection probability=0.448
B=0.5551, detection probability=0.452
B=0.5561, detection probability=0.404
B=0.5571, detection probability=0.410
B=0.5581, detection probability=0.440
B=0.5591, detection probability=0.448
B=0.5601, detection probability=0.386
B=0.5611, detection probability=0.422
B=0.5621, detection probability=0.484
B=0.5631, detection probability=0.406
B=0.5641, detection probability=0.482
B=0.5652, detection probability=0.424
B=0.5662, detection probability=0.430
B=0.5672, detection probability=0.488
B=0.5682, detection probability=0.436
B=0.5692, detection probability=0.450
B=0.5702, detection probability=0.466
B=0.5712, detection probability=0.422
B=0.5722, detection probability=0.466
B=0.5732, detection probability=0.454
B=0.5742, detection probability=0.454
B=0.5753, detection probability=0.484
B=0.5763, de

In [17]:
import numpy as np
import statsmodels.api as sm

def simulate(A=1, B=0.6136, C=10, D=1000):
    W = np.random.normal(0, 1, D)
    X = W + np.random.normal(0, B, D)
    Y = A * X - W + np.random.normal(0, C, D)
    return Y, X, W

def run_trials(n_trials=1000, A=1, B=0.6136, C=10, D=1000):
    count = 0
    for _ in range(n_trials):
        Y, X, W = simulate(A, B, C, D)
        XW = sm.add_constant(np.column_stack([X, W]))
        model = sm.OLS(Y, XW).fit()
        t_val = model.tvalues[1]  # t-stat for X
        count += (abs(t_val) > 1.96)
    return count / n_trials

print(f"Estimated detection probability: {run_trials():.3f}")

Estimated detection probability: 0.478


In [19]:
import numpy as np
import statsmodels.api as sm

def simulate(A=0.62, B=1, C=10, D=100):
    W = np.random.normal(0, 1, D)
    X = W + np.random.normal(0, B, D)
    Y = A * X - W + np.random.normal(0, C, D)
    return Y, X, W

def detection_prob(A, n_trials=2000, B=1, C=10, D=100):
    count = 0
    for _ in range(n_trials):
        Y, X, W = simulate(A, B, C, D)
        XW = sm.add_constant(np.column_stack([X, W]))
        model = sm.OLS(Y, XW).fit()
        t_val = model.tvalues[1]  # t-stat for X
        count += (abs(t_val) > 1.96)
    return count / n_trials

for A in [4.0, 0.5, 2.0, 1.0]:
    print(f"A={A:.2f}, detection probability={detection_prob(A):.3f}")

A=4.00, detection probability=0.972
A=0.50, detection probability=0.072
A=2.00, detection probability=0.509
A=1.00, detection probability=0.186
